In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=20, use_memory_fs=False)

import os
from ete3 import Tree
import random

from typing import Optional
from collections import defaultdict

import sys
sys.path.append("/groups/itay_mayrose/halabikeren/tmp/ploidb/data_processing/")
from check_tree_monophyly import get_largest_monophyletic_group

sys.path.append("/groups/itay_mayrose/halabikeren/tmp/plant_pollinator_inter/data_processing/name_resolution/")
from resolved_names_curator import ResolvedNamesCurator

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


/groups/itay_mayrose/halabikeren/miniconda3/envs/ppn/lib/python3.10/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
/groups/itay_mayrose/halabikeren/miniconda3/envs/ppn/lib/python3.10/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [2]:
resolve_tree = False
resolve_ccdb = False # ccdb taxonome name resolution is set to false
expand_tree = True

raw_ccdb_path = f"/groups/itay_mayrose/halabikeren/PloiDB/ccdb/all_data.csv" 
ccdb_path = f"/groups/itay_mayrose/halabikeren/PloiDB/ccdb/resolved_data_name_resolved_on_{'none' if not resolve_tree and not resolve_ccdb else ('only_ccdb' if not resolve_tree else 'ccdb_and_tree')}.csv"
ccdb_unresolved_names_path = "/groups/itay_mayrose/halabikeren/PloiDB/name_resolution/ccdb_unresolved_names.csv"
allotb_unresolved_names_path = "/groups/itay_mayrose/halabikeren/PloiDB/name_resolution/ALLOTB_tree_unresolved_names.csv"
allotb_tree_path = "/groups/itay_mayrose/halabikeren/PloiDB/trees/ALLOTB.tre" 

tree_resolved_names_path=f"/groups/itay_mayrose/halabikeren/PloiDB/name_resolution/processed_tree_resolved_names_name_resolution_on_{'none' if not resolve_tree and not resolve_ccdb else ('only_ccdb' if not resolve_tree else 'ccdb_and_tree')}.csv"
ccdb_resolved_names_path=f"/groups/itay_mayrose/halabikeren/PloiDB/name_resolution/processed_ccdb_resolved_names_name_resolution_on_{'none' if not resolve_tree and not resolve_ccdb else ('only_ccdb' if not resolve_tree else 'ccdb_and_tree')}.csv"
intersection_resolved_names_path=f"/groups/itay_mayrose/halabikeren/PloiDB/name_resolution/processed_intersection_resolved_names_{'none' if not resolve_tree and not resolve_ccdb else ('only_ccdb' if not resolve_tree else 'ccdb_and_tree')}.csv"

unresolved_tree_path = "/groups/itay_mayrose/halabikeren/PloiDB/trees/ALLOTB.tre"
resolved_tree_path = f"/groups/itay_mayrose/halabikeren/PloiDB/trees/resolved_ALLOTB_name_resolution_on_{'none' if not resolve_tree and not resolve_ccdb else ('only_ccdb' if not resolve_tree else 'ccdb_and_tree')}.nwk"
selected_tree_leaves_path = f"/groups/itay_mayrose/halabikeren/PloiDB/trees/selected_ALLOTB_original_names_name_resolution_on_{'none' if not resolve_tree and not resolve_ccdb else ('only_ccdb' if not resolve_tree else 'ccdb_and_tree')}.csv"

unresolved_ccdb_path = "/groups/itay_mayrose/halabikeren/PloiDB/ccdb/all_data.csv"
resolved_ccdb_path = f"/groups/itay_mayrose/halabikeren/PloiDB/ccdb/resolved_data_name_resolved_on_{'only_ccdb' if not resolve_tree else 'ccdb_and_tree'}.csv"

resolved_tree_path_with_additions =  f"/groups/itay_mayrose/halabikeren/PloiDB/trees/resolved_ALLOTB_name_resolution_on_{'ccdb_and_tree' if resolve_ccdb and resolve_tree else ('only_ccdb' if resolve_ccdb else 'none')}_with_added_ccdb_names.nwk"

In [3]:
ccdb_unresolved_names = pd.read_csv(ccdb_unresolved_names_path)
tree_unresolved_names = pd.read_csv(allotb_unresolved_names_path)

In [4]:
def process_tree(path: str) -> Tree:
    tree = Tree(path, format=1)
    names = set()
    for leaf in tree.get_leaves():
        leaf_name = leaf.name.lower().replace("_", " ")
        if leaf_name.endswith(" "):
            leaf_name = leaf_name[:-1]
        if leaf_name in names:
            print(f"{leaf_name} already in tree")
            leaf.detach()
        else:
            leaf.name = leaf_name
            names.add(leaf_name)
    return tree
    
unresolved_tree = process_tree(unresolved_tree_path)

In [5]:
def get_taxonome_name_resolution(ccdb_path: str) -> pd.DataFrame:
    ccdb = pd.read_csv(raw_ccdb_path)
    ccdb_resolved_names = ccdb[["original_name", "matched_name", "resolved_name", "genus", "family"]]
    for col in ccdb_resolved_names:
        ccdb_resolved_names[col] = ccdb_resolved_names[col].str.lower()
    if not "corrected_matched_name" in ccdb_resolved_names.columns:
        ccdb_resolved_names["corrected_matched_name"] = ccdb_resolved_names["matched_name"].apply(lambda name: ResolvedNamesCurator.fix_name(name) if pd.notna(name) else np.nan)
    if not "corrected_resolved_name" in ccdb_resolved_names.columns:
        ccdb_resolved_names["corrected_resolved_name"] = ccdb_resolved_names["resolved_name"].apply(lambda name: ResolvedNamesCurator.fix_name(name) if pd.notna(name) else np.nan)
    return ccdb_resolved_names

if resolve_ccdb:
    ccdb_resolved_names = pd.read_csv(ccdb_resolved_names_path)
else:
    ccdb_resolved_names = get_taxonome_name_resolution(ccdb_path)
    ccdb_resolved_names.to_csv(ccdb_resolved_names_path, index=False)

/tmp/ipykernel_23551/1191046547.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccdb_resolved_names[col] = ccdb_resolved_names[col].str.lower()
/tmp/ipykernel_23551/1191046547.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccdb_resolved_names["corrected_matched_name"] = ccdb_resolved_names["matched_name"].apply(lambda name: ResolvedNamesCurator.fix_name(name) if pd.notna(name) else np.nan)
/tmp/ipykernel_23551/1191046547.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [6]:
if resolve_ccdb and resolve_tree:
    intersection_resolved_names = pd.read_csv(intersection_resolved_names_path)
    print(f"# ccdb resolved names that are present in the original tree = {len(intersection_resolved_names.loc[intersection_resolved_names.mapped_name == intersection_resolved_names.tree_corrected_resolved_name].mapped_name.unique()):,}")
    print(f"# ccdb matched names that are present in the original tree = {len(intersection_resolved_names.loc[intersection_resolved_names.mapped_name == intersection_resolved_names.tree_corrected_matched_name].mapped_name.unique()):,}")
else:
    intersection_resolved_names = pd.DataFrame({"tree_original_name": unresolved_tree.get_leaf_names()})
    intersection_with_ccdb_resolved_names = intersection_resolved_names.merge(ccdb_resolved_names, left_on="tree_original_name", right_on="corrected_resolved_name", how="inner").drop_duplicates("tree_original_name")
    intersection_with_ccdb_resolved_names["mapped_name"] = intersection_with_ccdb_resolved_names["corrected_resolved_name"]
    print(f"# ccdb resolved names that are present in the original tree = {len(intersection_with_ccdb_resolved_names.mapped_name.unique()):,}")
    intersection_with_ccdb_matched_names = intersection_resolved_names.merge(ccdb_resolved_names, left_on="tree_original_name", right_on="corrected_matched_name", how="inner").drop_duplicates("tree_original_name")
    intersection_with_ccdb_matched_names["mapped_name"] = intersection_with_ccdb_matched_names["corrected_matched_name"]
    print(f"# ccdb resolved names that are present in the original tree = {len(intersection_with_ccdb_matched_names.mapped_name.unique()):,}")
    intersection_resolved_names = pd.concat([intersection_with_ccdb_resolved_names, intersection_with_ccdb_matched_names]).drop_duplicates("tree_original_name", keep="first") 
    intersection_resolved_names.rename(columns={"original_name": "ccdb_original_name"}, inplace=True)
    intersection_resolved_names.to_csv(intersection_resolved_names_path, index=False)

print(f"# ccdb covered records (by resolved or matched name) = {intersection_resolved_names.shape[0]:,} spanning across {len(intersection_resolved_names.mapped_name.unique()):,} names")

# ccdb resolved names that are present in the original tree = 51,413
# ccdb resolved names that are present in the original tree = 51,944
# ccdb covered records (by resolved or matched name) = 56,623 spanning across 56,623 names


Select representative leafs for groups of leafs mapped to the same resolved name

In [7]:
def add_mapped_names(tree: Tree, resolved_names: pd.DataFrame):
    orig_to_mapped = resolved_names.set_index("tree_original_name")["mapped_name"].to_dict()
    for leaf in tree.get_leaves():
        mapped_name = orig_to_mapped[leaf.name]
        leaf.add_feature(pr_name="mapped_name", pr_value=mapped_name)


def get_phylogenetic_centroid(members: list[str], tree: Tree) -> tuple[str, float]:
    pairwise_distances = np.zeros(shape=(len(members), len(members)))
    for i in range(len(members)):
        for j in range(i+1, len(members)):
            dist = tree.get_distance(members[i], members[j], topology_only=False)
            pairwise_distances[i, j] = dist
            pairwise_distances[j, i] = dist
    centroid_index = pairwise_distances.sum(axis=1).argmin()
    centroid = members[centroid_index]
    return centroid, pairwise_distances[centroid_index].mean()


def select_orig_name(record: pd.Series, tree: Tree, ccdb_orig_names: list[str]) -> tuple[str, float]:
    mapped_name = record.mapped_name
    orig_names = record.original_names
    repr_name, repr_score = random.choice(orig_names), np.Inf
    if len(orig_names) <= 2:
        return orig_names[0], 0 if len(orig_names)==1 else tree.get_distance(orig_names[0],orig_names[1])
    

    is_monophyletic, clade_type, monophyly_violators = tree.check_monophyly(values=orig_names, target_attr="name") 
    if is_monophyletic:
        return random.choice(orig_names), 0
    else:
        try:
            centroid_repr, centroid_mean_dist = get_phylogenetic_centroid(members=orig_names, tree=tree)
        except Exception as e:
            print(f"failed to find centroid for {mapped_name} due to error {e}")
            exit(1)
        return centroid_repr, centroid_mean_dist

    return repr_name, repr_score
  
    
def resolve_tree(tree: Tree, resolved_names: pd.DataFrame, ccdb_orig_names: list[str], resolved_to_selected_orig_path: str) -> Tree:
    resolved_tree = tree.copy()
    print(f"# original leaves = {len(resolved_tree.get_leaf_names()):,}")
    leaves_to_keep = resolved_names.tree_original_name.tolist()
    resolved_tree.prune(leaves_to_keep, preserve_branch_length=True)
    add_mapped_names(tree=resolved_tree, resolved_names=resolved_names)
    print(f"# leaves after prunning of unmapped names to ccdb = {len(resolved_tree.get_leaf_names()):,}")
    if os.path.exists(resolved_to_selected_orig_path):
        mapped_to_orig = pd.read_csv(resolved_to_selected_orig_path)
    else:
        mapped_to_orig = resolved_names.groupby("mapped_name")["tree_original_name"].apply(lambda names: names.unique().tolist()).reset_index().rename(columns={"tree_original_name": "original_names"})
        mapped_to_orig[["selected_original_name", "selected_original_mean_dist_from_rest"]] = mapped_to_orig.parallel_apply(lambda record: select_orig_name(record=record, tree=resolved_tree, ccdb_orig_names=ccdb_orig_names), axis=1, result_type="expand")
    mapped_to_orig.drop_duplicates("mapped_name", keep="first", inplace=True)
    mapped_to_orig.to_csv(resolved_to_selected_orig_path, index=False)
    resolved_tree.prune(mapped_to_orig["selected_original_name"].tolist(), preserve_branch_length=True)
    print(f"# leaves after removal of leaves with identical resolved name = {len(resolved_tree.get_leaf_names()):,}")
    for leaf in resolved_tree.get_leaves():
        leaf.name = leaf.mapped_name
    return resolved_tree


resolved_tree = resolve_tree(tree=unresolved_tree, 
                             resolved_names=intersection_resolved_names, 
                             ccdb_orig_names=ccdb_resolved_names.original_name.tolist(), 
                             resolved_to_selected_orig_path=selected_tree_leaves_path)
resolved_tree.write(outfile=resolved_tree_path)

# original leaves = 353,185
# leaves after prunning of unmapped names to ccdb = 56,623
# leaves after removal of leaves with identical resolved name = 56,623


In [8]:
def are_diff_prefices(mapped_name: str, names: list[str]) -> bool:
    prefices = set()
    mapped_prefix = mapped_name.split(" ")[0]
    for name in names:
        if name.split(" ")[0] != mapped_prefix:
            return True
    return False

selected_tree_leaves = pd.read_csv(selected_tree_leaves_path).sort_values("selected_original_mean_dist_from_rest", ascending=False)
selected_tree_leaves["original_maped_names_diff_prefices"] = selected_tree_leaves[["mapped_name", "original_names"]].apply(lambda record: are_diff_prefices(record.mapped_name, record.original_names), axis=1)
selected_tree_leaves["num_original_names"] = selected_tree_leaves.original_names.apply(lambda x: len(x))
selected_tree_leaves.sort_values("num_original_names", ascending=False, inplace=True)
selected_tree_leaves.loc[selected_tree_leaves.original_maped_names_diff_prefices].head()

,mapped_name,original_names,selected_original_name,selected_original_mean_dist_from_rest,original_maped_names_diff_prefices,num_original_names
51195,streptocarpus beampingaratrensis subsp. beampi...,['streptocarpus beampingaratrensis subsp. beam...,streptocarpus beampingaratrensis subsp. beampi...,0,True,62
39370,pelargonium antidysentericum subsp. antidysent...,['pelargonium antidysentericum subsp. antidyse...,pelargonium antidysentericum subsp. antidysent...,0,True,56
31121,leucanthemum coronopifolium subsp. ceratophyll...,['leucanthemum coronopifolium subsp. ceratophy...,leucanthemum coronopifolium subsp. ceratophyll...,0,True,55
19539,echinocereus triglochidiatus subsp. triglochid...,['echinocereus triglochidiatus subsp. trigloch...,echinocereus triglochidiatus subsp. triglochid...,0,True,55
40593,phyllanthus juglandifolius subsp. juglandifolius,['phyllanthus juglandifolius subsp. juglandifo...,phyllanthus juglandifolius subsp. juglandifolius,0,True,52


Now expand the tree with ccdb names that are absent from it

In [10]:
def get_mapped_name(record: pd.Series) -> Optional[str]:
    if pd.notna(record.resolved_name):
        return ResolvedNamesCurator.fix_name(record.resolved_name)
    elif pd.notna(record.matched_name):
        return ResolvedNamesCurator.fix_name(record.matched_name)
    return np.nan
       

if expand_tree:
    
    resolved_tree_with_addition = resolved_tree.copy()
    
    ccdb = pd.read_csv(raw_ccdb_path)

    if "mapped_name" not in ccdb.columns:
        ccdb["mapped_name"] = ccdb[["matched_name", "resolved_name"]].apply(get_mapped_name, axis=1)
    else:
        ccdb.loc[ccdb.mapped_name.isna(), "mapped_name"] = ccdb.loc[ccdb.mapped_name.isna()][["corrected_matched_name", "corrected_resolved_name"]].apply(lambda record: record.corrected_resolved_name if pd.notna(record.corrected_resolved_name) else record.corrected_matched_name, axis=1)
    
    print(f"# ccdb records without mapped names = {ccdb.loc[ccdb.mapped_name.isna()].shape[0]:,}")
    
    ccdb_names_not_in_tree = set(ccdb.dropna(subset=["mapped_name"]).mapped_name.str.lower().unique()) - set(resolved_tree_with_addition.get_leaf_names())
    print(f"# ccdb mapped names that are absent from the tree = {len(ccdb_names_not_in_tree):,}")

    tree_genera = set([name.split(" ")[0] for name in resolved_tree.get_leaf_names()])
    ccdb_names_to_add_to_tree = [name for name in ccdb_names_not_in_tree if name.split(" ")[0] in tree_genera]
    print(f"# ccdb names that can be added to the tree = {len(ccdb_names_to_add_to_tree):,}")

    ccdb_names_to_add_by_genus = pd.DataFrame({"mapped_name": ccdb_names_to_add_to_tree,
                                               "genus": [name.split(" ")[0] for name in ccdb_names_to_add_to_tree]}).groupby("genus")
    print(f"# genera to add ccdb direct children to {len(ccdb_names_to_add_by_genus.groups.keys()):,}")
    
    genus_to_names = defaultdict(list)
    for leaf_name in resolved_tree_with_addition.get_leaf_names():
        genus = leaf_name.split(" ")[0]
        genus_to_names[genus].append(leaf_name)

    print(f"computing lca per genus across {len(ccdb_names_to_add_by_genus.groups.keys()):,} genera")
    genus_to_ancestor = dict()
    for genus in ccdb_names_to_add_by_genus.groups.keys():
        genus_to_ancestor[genus] = resolved_tree_with_addition.get_common_ancestor(genus_to_names[genus])
    
    print(f"adding missing species under lca per genus across {len(genus_to_ancestor):,} genera")
    for genus in genus_to_ancestor:
        ancestor = genus_to_ancestor[genus]
        names = set(ccdb_names_to_add_by_genus.get_group(genus)["mapped_name"].tolist()) - set(ancestor.get_leaf_names())
        print(f"adding to genus {genus} {len(names):,} names")
        # leaves_dist = {np.round(ancestor.get_distance(leaf_name),3) for leaf_name in ancestor.get_leaf_names()}
        # assert(len(leaves_dist) == 1)
        # print(f"tree is ultrametric")
        time_to_leaf = ancestor.get_distance(ancestor.get_leaf_names()[0])
        for name in names:
            leaf = ancestor.add_child(name=name, dist=time_to_leaf)

    print(f"# leafs in new tree = {len(resolved_tree.get_leaf_names()):,}")
    resolved_tree_with_addition.write(outfile=resolved_tree_path_with_additions)

# ccdb records without mapped names = 5
# ccdb mapped names that are absent from the tree = 27,509
# ccdb names that can be added to the tree = 19,467
# genera to add ccdb direct children to 2,822
computing lca per genus across 2,822 genera
adding missing species under lca per genus across 2,822 genera
adding to genus aaronsohnia 2 names
adding to genus abelia 5 names
adding to genus abelmoschus 3 names
adding to genus abies 5 names
adding to genus abrotanella 2 names
adding to genus abrus 2 names
adding to genus abutilon 15 names
adding to genus acacia 86 names
adding to genus acaena 1 names
adding to genus acalypha 3 names
adding to genus acampe 2 names
adding to genus acanthophyllum 2 names
adding to genus acanthorrhinum 1 names
adding to genus acer 23 names
adding to genus acetosa 6 names
adding to genus achillea 65 names
adding to genus achimenes 1 names
adding to genus achlys 1 names
adding to genus achyranthes 5 names
adding to genus achyrocline 2 names
adding to genus acianther